In [1]:
import yfinance as yf


In [2]:
import yfinance as yf
import pandas as pd

tickers = ["SPY", "QQQ", "IWM", "TLT", "GLD", "XLE", "XLF", "EEM", "HYG", "DBC"]

raw = yf.download(
tickers=tickers,
period="60d",
interval="2m",
group_by="ticker",
auto_adjust=True,
threads=True)




[*********************100%***********************]  10 of 10 completed


In [3]:
def clean_panel(raw):
    fields = ["Open", "High", "Low", "Close", "Volume"]
    df = raw.dropna(how="all")
    df = df.loc[:, pd.IndexSlice[:, fields]]
    df = df.sort_index()
    if df.index.tz is None:
        df = df.tz_localize("America/New_York", nonexistent="shift_forward", ambiguous="NaT")
    df = df.between_time("09:30", "16:00")
    df = df.groupby(axis=1, level=0).ffill(limit=2)
    df = df.dropna(how="all")
    return df


In [4]:
clean =  clean_panel(raw)

C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\79703078.py:9: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).ffill(limit=2)


In [5]:
clean.head()

Ticker,HYG,DBC,XLF,XLE,GLD,EEM,TLT,IWM,QQQ,SPY,...,HYG,DBC,XLF,XLE,GLD,EEM,TLT,IWM,QQQ,SPY
Price,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,...,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-08-29 13:30:00+00:00,80.989998,22.230000,53.900002,89.940002,314.720001,49.810001,86.779999,236.740005,574.640015,647.469971,...,101418.0,7354.0,313857.0,233124.0,255010.0,282656.0,2719303.0,1316151.0,2068681.0,2203540.0
2025-08-29 13:32:00+00:00,80.959999,22.209999,53.965000,90.040001,314.899994,49.794998,86.754997,236.649994,574.710022,647.695007,...,44015.0,1158.0,223081.0,124274.0,48818.0,267428.0,179742.0,240867.0,490322.0,539365.0
2025-08-29 13:34:00+00:00,80.964996,22.209999,53.980000,90.160004,314.910004,49.814999,86.754997,236.880005,574.294983,647.460022,...,340547.0,1243.0,770072.0,151293.0,56259.0,41051.0,226609.0,232714.0,342037.0,421286.0
2025-08-29 13:36:00+00:00,80.959999,22.209999,53.950001,90.195000,315.059998,49.799999,86.750000,236.570007,574.280029,647.320007,...,195429.0,829.0,1088469.0,121989.0,62549.0,94264.0,74366.0,158780.0,501444.0,560511.0
2025-08-29 13:38:00+00:00,80.955002,22.205000,53.950001,90.230003,315.051392,49.759998,86.719002,236.360001,574.049988,647.195007,...,725140.0,3000.0,336547.0,88551.0,270756.0,90746.0,68123.0,112074.0,505266.0,421337.0


In [6]:
import os

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)


raw.to_pickle("data/raw/etfs_1m_7d_raw.pkl")
clean.to_pickle("data/processed/etfs_1m_clean.pkl")


In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import os

os.makedirs("data/processed", exist_ok=True)

panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")


In [8]:
close = panel.loc[:, pd.IndexSlice[:, "Close"]]
volume = panel.loc[:, pd.IndexSlice[:, "Volume"]]
tickers = close.columns.get_level_values(0).unique()


In [9]:
# Returns
r1 = close.groupby(axis=1, level=0).pct_change(1)
r5 = close.groupby(axis=1, level=0).pct_change(5)

# Volatility
vol_s = r1.groupby(axis=1, level=0).rolling(60).std().droplevel(0)
vol_l = r1.groupby(axis=1, level=0).rolling(390).std().droplevel(0)



C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\1257453750.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  r1 = close.groupby(axis=1, level=0).pct_change(1)
C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\1257453750.py:2: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  r1 = close.groupby(axis=1, level=0).pct_change(1)
C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\1257453750.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  r5 = close.groupby(axis=1, level=0).pct_change(5)
C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\1257453750.py:3: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be r

In [10]:
vol_mean = volume.groupby(axis=1, level=0).rolling(390).mean().droplevel(0)
vol_norm = (volume / vol_mean.replace(0, np.nan)).clip(upper=10)



C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\2867922181.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  vol_mean = volume.groupby(axis=1, level=0).rolling(390).mean().droplevel(0)


In [11]:
proxy = "SPY" if "SPY" in tickers else tickers[0]

corr = pd.concat(
    {t: r1[(t,)].squeeze().rolling(390).corr(r1[(proxy,)].squeeze())
     for t in tickers},
    axis=1
)
corr.columns = pd.MultiIndex.from_product([corr.columns, ["corr_proxy"]])



C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\2341744871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  {t: r1[(t,)].squeeze().rolling(390).corr(r1[(proxy,)].squeeze())


In [12]:
vix = yf.download("^VIX", period="7d", interval="1m", auto_adjust=True)[["Close"]]
vix.columns = ["VIX_Close"]
if vix.index.tz is None:
    vix.index = vix.index.tz_localize("America/New_York", nonexistent="shift_forward", ambiguous="NaT")
else:
    vix.index = vix.index.tz_convert("America/New_York")

vix = vix.reindex(close.index).ffill()

vix_ret = vix["VIX_Close"].pct_change()
vix_vol = vix_ret.rolling(60).std()

vix_feature = pd.DataFrame({"vix_ret": vix_ret, "vix_vol": vix_vol})


[*********************100%***********************]  1 of 1 completed


In [13]:
vix_blocks = []
for t in tickers:
    block = pd.concat({t: vix_feature}, axis=1)
    vix_blocks.append(block)

vix_multi = pd.concat(vix_blocks, axis=1)



In [14]:
def compute_turbulence(r1, window=390):
    turb = []
    idx = r1.index
    for i in range(window, len(r1)):
        hist = r1.iloc[i - window:i]
        mu = hist.mean().values
        cov = hist.cov().values
        if np.linalg.det(cov) <= 0:
            turb.append(np.nan)
            continue
        diff = r1.iloc[i].values - mu
        m_dist = np.sqrt(diff.T @ np.linalg.inv(cov) @ diff)
        turb.append(m_dist)
    turb = pd.Series(turb, index=idx[window:])
    return turb.reindex(idx)


In [15]:
r1_all = close.xs("Close", level=1, axis=1).pct_change()
turbulence = compute_turbulence(r1_all)

turb_blocks = []
for t in tickers:
    turb_blocks.append(pd.concat({t: pd.DataFrame({"turbulence": turbulence})}, axis=1))
turb_multi = pd.concat(turb_blocks, axis=1)



C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\688292144.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  r1_all = close.xs("Close", level=1, axis=1).pct_change()


In [16]:
feat = pd.concat(
    {
        "ret_1": r1,
        "ret_5": r5,
        "vol_s": vol_s,
        "vol_l": vol_l,
        "vol_norm": vol_norm,
        "corr_proxy": corr
    },
    axis=1
).swaplevel(0, 1, axis=1).sort_index(axis=1)


In [17]:
feat = pd.concat([feat, vix_multi, turb_multi], axis=1).sort_index(axis=1)


In [18]:
feat = feat.replace([np.inf, -np.inf], np.nan).dropna(how="all")
feat.to_pickle("data/processed/etfs_1m_features.pkl")
print("Saved", feat.shape)


Saved (1890, 90)


In [19]:
print(type(feat.columns))
print(feat.columns[:10])


<class 'pandas.core.indexes.base.Index'>
Index([('DBC', 'corr_proxy', 'corr_proxy'),
                 ('DBC', 'ret_1', 'Close'),
                 ('DBC', 'ret_5', 'Close'),
                     ('DBC', 'turbulence'),
                        ('DBC', 'vix_ret'),
                        ('DBC', 'vix_vol'),
                 ('DBC', 'vol_l', 'Close'),
             ('DBC', 'vol_norm', 'Volume'),
                 ('DBC', 'vol_s', 'Close'),
       ('EEM', 'corr_proxy', 'corr_proxy')],
      dtype='object')


In [20]:
%%writefile src/env/portfolio_env.py
import numpy as np
import pandas as pd

class PortfolioEnv:
    def __init__(self, prices, start_equity=1_000_000):
        self.prices = prices
        self.close = prices.loc[:, pd.IndexSlice[:, "Close"]]
        self.tickers = self.close.columns.get_level_values(0).unique()
        self.start_equity = start_equity
        self.reset()

    def reset(self):
        self.t = 1
        self.equity = float(self.start_equity)
        self.w = np.zeros(len(self.tickers))
        return self._get_obs()

    def _get_obs(self):
        return {
            "t": self.t,
            "weights": self.w.copy(),
            "equity": self.equity
        }

    def step(self, w_target):
        w_target = np.clip(w_target, 0, 1)
        w_target = w_target / w_target.sum() if w_target.sum() > 0 else self.w

        r_next = (
            self.close.iloc[self.t + 1].values /
            self.close.iloc[self.t].values - 1.0
        )

        portfolio_ret = np.dot(w_target, r_next)
        self.equity *= (1 + portfolio_ret)
        self.w = w_target
        self.t += 1

        reward = portfolio_ret
        done = self.t >= len(self.close) - 1

        return self._get_obs(), reward, done, {"portfolio_ret": portfolio_ret}


Overwriting src/env/portfolio_env.py


In [21]:
import pandas as pd

panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")
close = panel.loc[:, pd.IndexSlice[:, "Close"]]
print(close.isna().sum().sum())


640


In [22]:
panel = panel.groupby(axis=1, level=0).ffill().bfill()


C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\2944265229.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  panel = panel.groupby(axis=1, level=0).ffill().bfill()


In [23]:
panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")

# Forward-fill small gaps per ticker
panel = panel.groupby(axis=1, level=0).ffill(limit=5)
panel = panel.groupby(axis=1, level=0).bfill(limit=5)

# Drop any rows that are still all-NaN
panel = panel.dropna(how="all")

# Verify
print("NaNs remaining:", panel.isna().sum().sum())

# Save cleaned version
panel.to_pickle("data/processed/etfs_1m_clean.pkl")


NaNs remaining: 3200


C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\252775637.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  panel = panel.groupby(axis=1, level=0).ffill(limit=5)
C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\252775637.py:5: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  panel = panel.groupby(axis=1, level=0).bfill(limit=5)


In [24]:
panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")

# Fill per ticker forward and backward across entire session
panel = panel.groupby(axis=1, level=0).apply(lambda g: g.ffill().bfill())

# Drop any all-NaN rows left
panel = panel.dropna(how="all")

print("NaNs remaining:", panel.isna().sum().sum())

panel.to_pickle("data/processed/etfs_1m_clean.pkl")


NaNs remaining: 0


C:\Users\mithi\AppData\Local\Temp\ipykernel_19428\1271599181.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  panel = panel.groupby(axis=1, level=0).apply(lambda g: g.ffill().bfill())


In [25]:
if panel.columns.nlevels == 1:
    new_cols = []
    for c in panel.columns:
        parts = str(c).split("_", 1)
        if len(parts) == 2:
            new_cols.append((parts[0], parts[1]))
        else:
            new_cols.append((parts[0], "Close"))
    panel.columns = pd.MultiIndex.from_tuples(new_cols, names=["Ticker", "Field"])


In [26]:
panel.to_pickle("data/processed/etfs_1m_clean.pkl")


In [27]:
panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")
print(panel.columns[:10])
print(panel.columns.nlevels)


MultiIndex([('DBC', 'DBC',   'Open'),
            ('DBC', 'DBC',   'High'),
            ('DBC', 'DBC',    'Low'),
            ('DBC', 'DBC',  'Close'),
            ('DBC', 'DBC', 'Volume'),
            ('EEM', 'EEM',   'Open'),
            ('EEM', 'EEM',   'High'),
            ('EEM', 'EEM',    'Low'),
            ('EEM', 'EEM',  'Close'),
            ('EEM', 'EEM', 'Volume')],
           names=['Ticker', 'Ticker', 'Price'])
3


In [28]:
panel = pd.read_pickle("data/processed/etfs_1m_clean.pkl")

# Drop the duplicate middle level
if panel.columns.nlevels == 3:
    panel.columns = panel.columns.droplevel(1)

# Rename the third level to 'Field'
panel.columns.set_names(["Ticker", "Field"], inplace=True)

print(panel.columns.nlevels)
print(panel.columns.levels)


2
[['DBC', 'EEM', 'GLD', 'HYG', 'IWM', 'QQQ', 'SPY', 'TLT', 'XLE', 'XLF'], ['Close', 'High', 'Low', 'Open', 'Volume']]


In [29]:
panel.to_pickle("data/processed/etfs_1m_clean.pkl")


In [37]:
%%writefile src/utils/costs.py
import numpy as np
import pandas as pd

def spread_proxy(high, low):
    rng = (high - low).clip(lower=0)
    mid = 0.5 * (high + low)
    sp = (rng / mid).fillna(0.0)
    return 0.25 * sp

def realized_vol(close, win=60):
    r = close.pct_change()
    return r.rolling(win).std().fillna(0.0)

def participation(dollar_trade, price, volume):
    dollar_bar = (price * volume).replace(0, np.nan)
    return (dollar_trade / dollar_bar).fillna(0.0).clip(lower=0.0)

def exec_price(side, mid, half_spread, sigma, dt_min, part, k=0.6, lam=2e-4):
    drift = k * sigma * np.sqrt(max(dt_min, 1.0))
    impact = lam * (part ** 2)
    if side > 0:
        px = mid * (1.0 + half_spread + drift + impact)
    elif side < 0:
        px = mid * (1.0 - half_spread - drift - impact)
    else:
        px = mid
    return px

def turnover_l1(w_new, w_old):
    """L1 turnover penalty: sum of absolute changes in portfolio weights."""
    return np.abs(w_new - w_old).sum()


Overwriting src/utils/costs.py


In [1]:
%%writefile src/env/portfolio_env.py
import numpy as np
import pandas as pd
from src.utils.costs import spread_proxy, realized_vol, participation, exec_price, turnover_l1

class PortfolioEnv:
    def __init__(self, prices, freq_min=1, start_equity=1_000_000, part_cap=0.05, k=0.6, lam=2e-4, gamma_bar=8.0, eta_turnover=0.001):
        self.prices = prices
        self.close = prices.loc[:, pd.IndexSlice[:, "Close"]]
        self.high = prices.loc[:, pd.IndexSlice[:, "High"]]
        self.low  = prices.loc[:, pd.IndexSlice[:, "Low"]]
        self.vol  = prices.loc[:, pd.IndexSlice[:, "Volume"]]
        self.mid  = self.close
        self.half_spread = spread_proxy(self.high, self.low)
        self.sigma = realized_vol(self.close, win=60)
        self.part_cap = part_cap
        self.k = k
        self.lam = lam
        self.freq_min = freq_min
        self.gamma = gamma_bar / (252 * 390)   # annualize risk cost
        self.eta_turnover = eta_turnover
        self.start_equity = start_equity
        self.tickers = self.close.columns.get_level_values(0).unique()
        self.reset()

    def reset(self):
        self.t = 1
        self.equity = float(self.start_equity)
        self.w = np.zeros(len(self.tickers))
        return self._obs()

    def _obs(self):
        return obs, reward, done, {"equity": float(self.equity), "portfolio_ret": portfolio_ret}


    def step(self, w_target):
        w_target = np.clip(w_target, 0, 1)
        if w_target.sum() > 0:
            w_target = w_target / w_target.sum()

        delta_w = w_target - self.w
        idx = self.close.index[self.t]
        mid = self.mid.loc[idx].values
        hs  = self.half_spread.loc[idx].values
        sg  = self.sigma.loc[idx].values
        vol = self.vol.loc[idx].values

        dollar_trade = np.abs(delta_w) * self.equity
        part = participation(pd.Series(dollar_trade), pd.Series(mid), pd.Series(vol)).values
        part = np.clip(part, 0, self.part_cap)
        side = np.sign(delta_w)

        exec_px = np.array([
            exec_price(s, m, h, sgm, self.freq_min, p, self.k, self.lam)
            for s, m, h, sgm, p in zip(side, mid, hs, sg, part)
        ])

        shares = (np.abs(delta_w) * self.equity) / np.maximum(mid, 1e-12)
        trade_cash = np.sum(shares * exec_px * side)

        dollar_pos = self.w * self.equity
        dollar_pos += shares * exec_px * side
        self.equity -= np.abs(trade_cash)
        self.cash = max(self.start_equity - np.sum(dollar_pos), 0.0)

        idx_next = self.close.index[min(self.t + 1, len(self.close) - 1)]
        next_px = self.mid.loc[idx_next].values
        dollar_pos = dollar_pos * (next_px / np.maximum(mid, 1e-12))
        self.equity = np.sum(dollar_pos) + self.cash
        self.w = np.clip(dollar_pos / self.equity, 0, 1)

        # risk penalty
        var_diag = (self.sigma.loc[idx] ** 2).values
        risk_pen = 0.5 * self.gamma * float(np.dot(self.w, var_diag * self.w))

        # turnover penalty
        tvr = turnover_l1(self.w, self.w - delta_w)
        tvr_pen = self.eta_turnover * tvr

        # reward
        prev_idx = self.close.index[self.t - 1]
        r_bar = self.mid.loc[idx_next].values / np.maximum(self.mid.loc[prev_idx].values, 1e-12) - 1.0
        pnl_ret = float(np.dot(self.w, r_bar))
        reward = pnl_ret - risk_pen - tvr_pen

        reward = pnl_ret - risk_pen - tvr_pen

        info = {
        "pnl_ret": pnl_ret,
        "risk_pen": risk_pen,
        "tvr_pen": tvr_pen,
        "equity": float(self.equity)
        }
        self.t += 1
        done = self.t >= len(self.close) - 1
        return self._obs(), reward, done, info




Overwriting src/env/portfolio_env.py


In [38]:
%%writefile src/env/portfolio_gym.py
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from src.env.portfolio_env import PortfolioEnv

class PortfolioGym(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, prices, feature_df=None, start_equity=1_000_000):
        super().__init__()
        self.core = PortfolioEnv(prices, start_equity=start_equity)
        self.feature_df = feature_df
        self.tickers = self.core.tickers
        self.n_assets = len(self.tickers)

        # Observation: weights + features (can be expanded later)
        obs_dim = self.n_assets * 2
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)

        # Action: target weights (0 to 1)
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(self.n_assets,), dtype=np.float32)

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        obs_core = self.core.reset()
        obs = self._build_obs(obs_core)
        return obs, {}

    def _build_obs(self, core_obs):
        w = core_obs["weights"]
        if self.feature_df is not None:
            f = self.feature_df.iloc[self.core.t].xs(level=0, axis=1, drop_level=False).values.flatten()
            return np.concatenate([w, f])
        else:
            return np.concatenate([w, w])  # placeholder if no features

    def step(self, action):
        obs_core, reward, done, info = self.core.step(action)
        obs = self._build_obs(obs_core)
        truncated = False
        return obs, reward, done, truncated, info

    def render(self):
        print(f"Step {self.core.t}, Equity {self.core.equity:.2f}")


Writing src/env/portfolio_gym.py


In [4]:
!git add notebook_1_*.ipynb test.ipynb

